In [1]:
# 이런 저런 모델을 만들어서 계속 테스트 세트로만 평가하면 결국 테스트 세트에 잘 맞는 모델이 만들어지는 거 아닌가?
# 테스트 세트로 일반화 성능을 올바르게 예측하려면 테스트 세트는 1번만 사용
# 결정 트리의 하이퍼 파라미터 튜닝은 어떻게?

## 검증세트 
- 테스트 세트를 사용하지 않고 과대적합, 과소적합 판단하기 위해서는 훈련세트를 또 나누어 검증세트를 만든다
- 즉, 훈련세트 60%, 검증세트 20%, 훈련세트 20%임
- 데이터가 아주 많으면 몇%만 떼도 됨

In [2]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
# 넘파이 배열로 바꾸기

In [4]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
data, target, test_size=0.2, random_state=42
)
# 훈련, 테스트 세트 나누기 test_size default = 25%

In [9]:
train_input.shape, test_input.shape

((5197, 3), (1300, 3))

In [5]:
# 훈련세트에서 모델 훈련하고 검증 세트로 모델 평가
# 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고름
# 그다음 매개변수를 사용해 훈련세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련
# 마지막 테스트 세트에서 최종 점수를 평가

In [7]:
# 검증 세트 만들기
sub_input, val_input, sub_target, val_target = train_test_split(
train_input, train_target, test_size=0.2, random_state=42
)
#훈련세트를 또다시 검증세트 20% 떼어내고 나머지를 훈련세트로 만들기

In [8]:
# 크기 확인
sub_input.shape, val_input.shape
# 훈련세트 5197개 중에서 4157개와 1040개로 나눔

((4157, 3), (1040, 3))

In [10]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
# min_impurity_decrease어떤 노드의 정보이득 x(노드의 샘플 수) / (전체 샘플 수)값이 이 매개변수보다 작으면 더 이상 분할하지 않음
# min_impurity_decrease최소 불순도

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [11]:
dt.score(sub_input, sub_target)

0.9971133028626413

In [12]:
dt.score(val_input, val_target)

0.864423076923077

In [13]:
# 훈련세트>검증세트로 과대적합 문제

In [14]:
# 매개 변수를 바꿔서 더 좋은 모델 찾기

In [16]:
# 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어짐
# 검증 세트를 너무 조금 떼어놓으면 검증 점수가 들쭉날쭉하고 불안정할 것임
# 이럴 때 교차 검증을 이요하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터 사용
# 보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용
# 데이터의 80~90%까지 훈련에 사용할 수 있음
# 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각할 수 있음

In [17]:
# 교차검증 함수 사용 
# 전처럼 직접 검증세트 떼어내지 않고 훈련 세트 전체를 검증 함수에 전달
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)

In [18]:
scores

{'fit_time': array([0.00999975, 0.01000166, 0.00900173, 0.00799894, 0.00800157]),
 'score_time': array([0.00200057, 0.00100088, 0.00100207, 0.00099993, 0.00100136]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [ ]:
# fit_time, 모델을 훈련하는 시간, score_time 검증하는 시간, test_score는 5-폴드 교차 검증 수행
# cv매개변수에서 폴드 수를 바꿀 수 있음

In [19]:
# 교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수 평균하여 얻음
# 최상의 검증 점수
import numpy as np
np.mean(scores['test_score'])

0.855300214703487

In [ ]:
# cross_validate()는 훈션 세트를 섞어 폴드를 나누지 않음
# 앞서 train_test_split() 함수로 전체 데이터 섞은 후 훈련 세트 준비했기 때문에 따로 섞을 필요가 없음
# 만약 교차 검증을 할 때 훈련 세트 섞으려면 분할기를 지정해야 함

## 훈련세트를 섞어 교차검증할 때 분할기 사용
- 분류 모델일 경우 StratifiedKFold
- 회귀 모델일 경우 KFold 

In [20]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
np.mean(scores['test_score'])

0.855300214703487

In [21]:
# 10-폴드 교차 검증 수행하려면 다음과 같이 작성
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8574181117533719

## 그리드 서치
### 하이퍼파라미터 튜닝
- 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾으면 안됨
- 두 매개변수를 동시에 바꿔가며 최적의 값 찾아야 함
- 파이썬의 for 반복문이나 그리드 서치를 사용할 수 있음

### 사이킷 런 그리드서치 클래스는 하이퍼파라미터 탐색과 교차 검증 한번에 수행, 별도로 cross_validate()함수 호출할 필요없음

In [26]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듬

In [28]:
# 그리드 서치에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체 만듬
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)
# n_jobs의 기본값은 1, -1은 시스템에 있는 모든 코어 사용
# 그리드 서치 cv 매개변수 기본값은 5이며 값마다 5-폴드 교차 검증 수행
# 5x5=25개 모델 훈련

In [29]:
gs.fit(train_input, train_target)
# 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 함
# 그리드서치는 편리하게도 25개 모델 중에서 가장 점수가 높은 모델의 매개변수 조합으로 전체 훈련세트에서 자동으로 모델 훈련
# 해당 모델은 gs객체의 best_estimator_ 속성에 저장되어 있음, 이 모델을 일반 결정 트리처럼 똑같이 사용

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [30]:
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9615162593804117

In [31]:
# 최적의 매개변수 확인
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [32]:
# 각 매개변수에서 수행한 5번의 교차 검증의 평균 점수
gs.cv_results_['mean_test_score']
# 첫번째 값이 가장큼

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

In [33]:
# 넘파이 argmax()함수를 사용하면 가장 큰 값의 인덱스 추출할 수 있음
# 가장 큰값의 인덱스로 params 키에 저장된 매개변수 출력할 수 있음
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


## 조금 더 복잡한 매개변수 조합 탐색

In [34]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), 
         'max_depth': range(5, 20, 1),
         'min_samples_split': range(2, 100, 10)
         }
#'min_impurity_decrease' 첫번째 매개변수 값에서 시작해서 두번째 매개변수에 도달할 때까지 세번째 매개변수를 계속 더한 배열 만듬 총 9개
# 'max_depth'range 함수는 5에서 20까지 1씩 증가하면서 15개 값을 만듬, min_samples_split은 2에서 100까지 10씩 증가하면서 10개의 값 aksema
# 모델의 갯수는 총 9개 x 15 x 10 = 1,350개임
# 기본 5-폴드 교차 검증 수행하므로 만들어지는 모델의 수는 6,750개가 됨
# n_jobs 매개변수를 -1로 설정하여 그리드 서치 진행

In [35]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [36]:
gs.best_params_
# 최상의 매개변수 조합

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [38]:
np.max(gs.cv_results_['mean_test_score'])
# 최상의 교차검증 점수 확인

0.8683865773302731

### 아쉬운점 :  탐색할 매개변수 간격을 0.0001 혹은 1로 설정했는데 이보다 더 좁거나 넓은 간격으로 시도할 수 없을까?
### 매개변수의 값이 수치이거나 값의 범위나 간격을 미리 정하기 어려울 수 있음, 너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간 오래 걸릴 수 있음

## 랜덤 서치
- 매개변수 값의 목록을 전달하는 것이 아닌, 매개변수를 샘플링할 수 있는 확률 분포 객체 전달

In [39]:
# 싸이파이에서 2개의 확률 분포 클래스를 임포트 하기
from scipy.stats import uniform, randint
# 주어진 범위에서 고르게 값을 뽑음, 이를 균등 분포에서 샘플링한다라고 말함
# radint는 정숫값을 뽑고, uniform은 실숫값을 뽑음
# 임의로 샘플링하여 실행할 때마다 결과가 다를 수 있음
# 난수 발생기랑 유사

In [40]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 7, 6, 2, 9, 6, 1, 7, 6, 7])

In [41]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99,  91, 112,  94,  85,  93,  97, 117, 106, 106], dtype=int64))

In [42]:
# 갯수가 늘어나니 0에서 9까지의 숫자가 어느정도 고르게 추출된 것을 볼 수 잇음
# uniform클래스의 사용법도 동일함, 0~1 사이에서 10개의 실수를 추출

In [43]:
# 0에서 1사이 실수 10개 추출
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.98204652, 0.62715438, 0.0793672 , 0.39848631, 0.59896551,
       0.95186893, 0.27795652, 0.40911723, 0.30269745, 0.88190177])

In [44]:
# 랜덤서치에 randint와 uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수 찾으라고 명령할 수 있음
# 샘플링 횟수는 시스템 자원이 허락하는 범위에서 최대한 크게 하기

In [46]:
# 매개변수 추가하려 랜덤서치 적용하기
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 
         'max_depth': randint(5, 20, 1),
         'min_samples_split': randint(2, 100, 10),
          'min_samples_leaf': randint(1, 25),
         }
# 샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정

In [47]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [ ]:
# 총 100번(n_iter 매개변수)을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합 찾음
# 앞의 그리드서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색

In [48]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004180034749718639, 'min_samples_leaf': 1, 'min_samples_split': 12}


In [49]:
# 최고의 교차 검증 점수
np.max(gs.cv_results_['mean_test_score'])

0.8693484859702376

In [50]:
# 최적의 모델 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))
# 테스트 세트 성능 확인

0.8615384615384616


In [51]:
# 테스트 세트 점수는 검증 세트에 대한 점수보다 조금 작은 것이 일반적

## 예제

In [52]:
# splitter='random'매개변수 추가하고 훈련
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random',random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [53]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004180034749718639, 'min_samples_leaf': 1, 'min_samples_split': 12}


In [54]:
# 최고의 교차 검증 점수
np.max(gs.cv_results_['mean_test_score'])

0.841640630784038

In [55]:
# 최적의 모델 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))
# 테스트 세트 성능 확인

0.7884615384615384


### 테스트 세트 성능이 떨어짐, splitter='best'로 각 노드에서 최선의 분할 찾아야 성능이 우수